gamma

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Oct  3 22:53:01 2023

@author: hkpen
"""
import numpy as np
import json
#import sys
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
#import numpy as np

D={}
mat_id_list= []
sg_number_list=[]
formulae_list=[]
band_gap_list=[]
nelements_list=[]
elements_list=[]
discovery_route_list=[]
formula_anonymous_list=[]
exfoliation_energy_list=[]
decomposition_energy_list=[]
crystal_list=[]
point_group_list=[]

open and read db file

In [2]:
with open('db.json') as f:
    for jsonobj in f:
        strDict=json.loads(jsonobj)
        mat_id_list.append(strDict['material_id'])
        sg_number_list.append(strDict['sg_number'])
        formulae_list.append(strDict['formula_pretty'])
        band_gap_list.append(strDict['bandgap'])
        nelements_list.append(strDict['nelements'])
        elements_list.append(strDict['elements'])
        discovery_route_list.append(strDict['discovery_process'])
        formula_anonymous_list.append(strDict['formula_anonymous'])
        if 'exfoliation_energy_per_atom' in strDict:
            exfoliation_energy_list.append(strDict['exfoliation_energy_per_atom'])
        else:
            exfoliation_energy_list.append(float("nan"))
        if 'decomposition_energy' in strDict:
            decomposition_energy_list.append(strDict['decomposition_energy'])
        else:
            decomposition_energy_list.append(float("nan"))
        crystal_list.append(strDict['spacegroup']['crystal_system'])
        point_group_list.append(strDict['spacegroup']['point_group'])

In [3]:
def calc_gamma_fingerprint(bands_dict, n_bins=32, e_range=[-4, 4]):
    band_energies = np.array(bands_dict["bands"]["1"]) - bands_dict["efermi"]
    band_energies_gamma = band_energies[:, 0]
    counts, edges = np.histogram(band_energies_gamma, bins=n_bins, range=e_range)
    density = counts / len(band_energies_gamma)
    return density

read data for each mat

In [4]:
fingerprint_length = 32
fingerprints = []

print(len(mat_id_list))

for i, name in enumerate(mat_id_list):
    
    BS_dir= '../FULL_MATPEDIA_DATA/bands/'
    filename_bands=BS_dir+name+'.json'
    if not os.path.isfile(filename_bands):
        print("No such file %s" % filename_bands)
        continue
    bands_dict=json.load(open(filename_bands))
    
    fingerprints.append(calc_gamma_fingerprint(bands_dict, n_bins=fingerprint_length))

6351
No such file ../FULL_MATPEDIA_DATA/bands/2dm-9.json
No such file ../FULL_MATPEDIA_DATA/bands/2dm-35.json
No such file ../FULL_MATPEDIA_DATA/bands/2dm-69.json
No such file ../FULL_MATPEDIA_DATA/bands/2dm-77.json
No such file ../FULL_MATPEDIA_DATA/bands/2dm-106.json
No such file ../FULL_MATPEDIA_DATA/bands/2dm-113.json
No such file ../FULL_MATPEDIA_DATA/bands/2dm-116.json
No such file ../FULL_MATPEDIA_DATA/bands/2dm-151.json
No such file ../FULL_MATPEDIA_DATA/bands/2dm-168.json
No such file ../FULL_MATPEDIA_DATA/bands/2dm-212.json
No such file ../FULL_MATPEDIA_DATA/bands/2dm-246.json
No such file ../FULL_MATPEDIA_DATA/bands/2dm-291.json
No such file ../FULL_MATPEDIA_DATA/bands/2dm-302.json
No such file ../FULL_MATPEDIA_DATA/bands/2dm-414.json
No such file ../FULL_MATPEDIA_DATA/bands/2dm-446.json
No such file ../FULL_MATPEDIA_DATA/bands/2dm-452.json
No such file ../FULL_MATPEDIA_DATA/bands/2dm-472.json
No such file ../FULL_MATPEDIA_DATA/bands/2dm-521.json
No such file ../FULL_MATPEDI

In [5]:
np.savetxt("gamma_fingerprints.csv", fingerprints, delimiter=",")